**Importation des bibliothèques**

In [18]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_classif
import time
from skimage.feature import graycomatrix, graycoprops
from skimage.feature import local_binary_pattern, hog


**Chargement du Data**

In [ ]:

# Chemins des répertoires
train_dir = r".\Train"
test_dir = r".\Test"
# Fonction pour charger les images et leurs labels
def load_data(directory):
    data = []
    labels = []
    for label in ["R", "O"]:
        path = os.path.join(directory, label)
        for file in os.listdir(path):
            img_path = os.path.join(path, file)
            img = cv2.imread(img_path)
            if img is not None:
                # Redimensionnez l'image pour uniformité
                img = cv2.resize(img, (128, 128))  # Taille arbitraire
                data.append(img)
                labels.append(0 if label == "R" else 1)  # 0 pour R, 1 pour O
    return np.array(data), np.array(labels)

# Chargement des données
X_train, y_train = load_data(train_dir)
X_test, y_test = load_data(test_dir)

print(f"Taille des données d'entraînement : {X_train.shape}")
print(f"Taille des labels d'entraînement : {y_train.shape}")
print(f"Taille des données de test : {X_test.shape}")
print(f"Taille des labels de test : {y_test.shape}")


Taille des données d'entraînement : (11816, 128, 128, 3)
Taille des labels d'entraînement : (11816,)
Taille des données de test : (5064, 128, 128, 3)
Taille des labels de test : (5064,)


**Préparation des modéles et leurs evaluations**

In [20]:
# Fonction pour entraîner et évaluer les modèles
def train_and_evaluate_models(X_train, X_test, y_train):
    # Dictionnaire pour stocker les modèles, les prédictions et les temps
    model_results = {}

    # 1. Modèle SVM (Support Vector Machine)
    svm1 = SVC(kernel='linear', C=1.0, random_state=42)
    svm_time_start = time.time()
    svm1.fit(X_train, y_train)
    svm_time_end = time.time()
    pred_svm = svm1.predict(X_test)
    model_results['SVM'] = {
        'model': svm1,
        'predictions': pred_svm,
        'time': svm_time_end - svm_time_start
    }

    # 2. Modèle Random Forest
    random_forest1 = RandomForestClassifier(n_estimators=100, random_state=42)
    rd_time_start = time.time()
    random_forest1.fit(X_train, y_train)
    rd_time_end = time.time()
    pred_rf = random_forest1.predict(X_test)
    model_results['Random Forest'] = {
        'model': random_forest1,
        'predictions': pred_rf,
        'time': rd_time_end - rd_time_start
    }

    # 3. Modèle XGBoost
    xgb_model1 = xgb.XGBClassifier(n_estimators=100, random_state=42)
    xgb_time_start = time.time()
    xgb_model1.fit(X_train, y_train)
    xgb_time_end = time.time()
    pred_xgb = xgb_model1.predict(X_test)
    model_results['XGBoost'] = {
        'model': xgb_model1,
        'predictions': pred_xgb,
        'time': xgb_time_end - xgb_time_start
    }

    # 4. Modèle KNN (K-Nearest Neighbors)
    knn1 = KNeighborsClassifier(n_neighbors=5)
    knn_time_start = time.time()
    knn1.fit(X_train, y_train)
    knn_time_end = time.time()
    pred_knn = knn1.predict(X_test)
    model_results['KNN'] = {
        'model': knn1,
        'predictions': pred_knn,
        'time': knn_time_end - knn_time_start
    }

    # Retourner les résultats sous forme de dictionnaire
    return model_results


# 1ére Méthode

### **1- Extraction des descripteurs :** 
* **Méthode Analyse de Texture: "LBP"**  

* **Techniques de Détection de Contours "Sobel Operator"**

In [21]:
def extract_features(image):
    # Conversion en niveaux de gris
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 1. Sobel Operator pour les gradients (détection des contours)
    grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)  # Gradient horizontal
    grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)  # Gradient vertical
    grad_magnitude = cv2.magnitude(grad_x, grad_y)  # Magnitude du gradient
    grad_hist = np.histogram(grad_magnitude.ravel(), bins=8, range=(0, 256))[0]
    grad_hist = grad_hist.astype("float")
    grad_hist /= (grad_hist.sum() + 1e-6)  # Normalisation

    # 2. Descripteur LBP (Local Binary Patterns)
    radius = 1  # Rayon autour du pixel central
    n_points = 8 * radius  # Nombre de voisins autour du pixel central
    lbp = local_binary_pattern(gray, n_points, radius, method='uniform')  # Calcul du LBP
    lbp_hist = np.histogram(lbp.ravel(), bins=59, range=(0, 59))[0]  # Histogramme des patterns LBP
    lbp_hist = lbp_hist.astype("float")
    lbp_hist /= (lbp_hist.sum() + 1e-6)  # Normalisation

    # Concaténer tous les descripteurs
    features = np.hstack([grad_hist, lbp_hist])
    return features
# Aplatir les images en un seul vecteur pour chaque image
def flatten_images(images):
    return images.reshape(images.shape[0], -1)

# Extraire les caractéristiques pour tout l'ensemble de données
def process_dataset(images):
    features = []
    for img in images:
        # Extraction des caractéristiques pour chaque image
        feature = extract_features(img)
        features.append(feature)

    # Convertir la liste des caractéristiques en un tableau 2D
    return np.array(features)
# Vérification des formes des données
def check_data_shape(X_train, X_test):
    print(f"Taille des caractéristiques d'entraînement : {X_train.shape}")
    print(f"Taille des caractéristiques de test : {X_test.shape}")
# Extraction des caractéristiques pour l'entraînement et les tests
X_train_features = process_dataset(X_train)
X_test_features = process_dataset(X_test)

# Vérification de la forme des données extraites
check_data_shape(X_train_features, X_test_features)
# Aplatir les caractéristiques
X_train_features_flat = flatten_images(X_train_features)
X_test_features_flat = flatten_images(X_test_features)


Taille des caractéristiques d'entraînement : (11816, 67)
Taille des caractéristiques de test : (5064, 67)


### **2-Entrainements  et évaluations des modéles sans Sélection des caracterstiques**

In [22]:
model_results_sans_selection = train_and_evaluate_models(X_train_features_flat, X_test_features_flat, y_train)
# Afficher les résultats pour chaque modèle
for model_name, result in model_results_sans_selection.items():
    print(f"****************************** Performance Evaluation of {model_name} ****************************************")
    print(classification_report(y_test, result['predictions'], target_names=["Recyclable", "Organique"]))
    print(f"Temps d'exécution {model_name} : {result['time']} seconds")

****************************** Performance Evaluation of SVM ****************************************
              precision    recall  f1-score   support

  Recyclable       0.70      0.73      0.71      2133
   Organique       0.80      0.78      0.79      2931

    accuracy                           0.76      5064
   macro avg       0.75      0.75      0.75      5064
weighted avg       0.76      0.76      0.76      5064

Temps d'exécution SVM : 31.23304581642151 seconds
****************************** Performance Evaluation of Random Forest ****************************************
              precision    recall  f1-score   support

  Recyclable       0.78      0.75      0.76      2133
   Organique       0.82      0.85      0.83      2931

    accuracy                           0.81      5064
   macro avg       0.80      0.80      0.80      5064
weighted avg       0.80      0.81      0.80      5064

Temps d'exécution Random Forest : 4.63299560546875 seconds
***********************

### **3- Sélection de caractéristiques (feature selection)**

C'est une étape cruciale dans le processus de prétraitement des données, en particulier pour les modèles d'apprentissage automatique. Elle permet d'améliorer les performances des modèles en réduisant la dimensionnalité, en supprimant les informations redondantes ou inutiles et en réduisant le risque de surajustement (overfitting).

#### **3.1. Sélection par Modèle (Embedded Method)**

**Objectif** : Évaluer l'importance des caractéristiques à partir du modèle d'apprentissage.

**Méthode** :
* Utiliser un modèle comme Random Forest pour obtenir l'importance de chaque caractéristique.
* Sélectionner les caractéristiques ayant la plus grande importance.

**1- Sélection par Modèle (Embedded Method) avec RandomForest**

In [23]:
# Sélection par Modèle (Embedded Method) avec RandomForest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_features_flat, y_train)
# Sélectionner les caractéristiques en fonction de l'importance calculée par RandomForest
select_from_model = SelectFromModel(rf_model, threshold="mean", max_features=20)  # Sélectionner les 20 meilleures caractéristiques
X_train_selected = select_from_model.transform(X_train_features_flat)
X_test_selected = select_from_model.transform(X_test_features_flat)
print(f"Taille des caractéristiques après sélection par modèle : {X_train_selected.shape}")

Taille des caractéristiques après sélection par modèle : (11816, 18)


**2- Entrainement  et evaluation des modéles avec la Séléction des caractérstiques avec la méthode embedded**

In [24]:
model_results_avec_selection = train_and_evaluate_models(X_train_selected, X_test_selected, y_train)
# Afficher les résultats pour chaque modèle
for model_name, result in model_results_avec_selection.items():
    print(f"****************************** Performance Evaluation of {model_name} ****************************************")
    print(classification_report(y_test, result['predictions'], target_names=["Recyclable", "Organique"]))
    print(f"Temps d'exécution {model_name} : {result['time']} seconds")

****************************** Performance Evaluation of SVM ****************************************
              precision    recall  f1-score   support

  Recyclable       0.70      0.73      0.71      2133
   Organique       0.80      0.78      0.79      2931

    accuracy                           0.76      5064
   macro avg       0.75      0.75      0.75      5064
weighted avg       0.76      0.76      0.76      5064

Temps d'exécution SVM : 26.558998823165894 seconds
****************************** Performance Evaluation of Random Forest ****************************************
              precision    recall  f1-score   support

  Recyclable       0.78      0.76      0.77      2133
   Organique       0.83      0.85      0.84      2931

    accuracy                           0.81      5064
   macro avg       0.81      0.80      0.80      5064
weighted avg       0.81      0.81      0.81      5064

Temps d'exécution Random Forest : 7.016042232513428 seconds
*********************

#### **3.2  Sélection par Test Statistique (Filter Method)**

Les tests statistiques, comme le test du chi carré (chi-squared test) ou **ANOVA**, sont utilisés pour évaluer la relation entre chaque caractéristique et la cible.

**Objectif** : Sélectionner les caractéristiques ayant une forte relation statistique avec la cible.

**Méthode** :

* Appliquer un test statistique pour mesurer la dépendance entre chaque caractéristique et la variable cible.
* Sélectionner les caractéristiques avec les valeurs de test les plus significatives.[texte du lien](https://)

**1- Sélection par Test Statistique (Filter Method) avec SelectKBest et ANOVA F-test**

In [25]:
# Sélection par Test Statistique (Filter Method) avec SelectKBest et ANOVA F-test
selector = SelectKBest(f_classif, k=20)  # Sélectionner les 20 meilleures caractéristiques
X_train_selected_filter = selector.fit_transform(X_train_features_flat, y_train)
X_test_selected_filter = selector.transform(X_test_features_flat)
print(f"Taille des caractéristiques après sélection par test statistique : {X_train_selected_filter.shape}")

Taille des caractéristiques après sélection par test statistique : (11816, 20)


c:\Users\debgn\anaconda3\envs\ml_env\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
 66] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\debgn\anaconda3\envs\ml_env\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


**2- Entrainement  et evaluation des modéles avec la Séléction des caractérstiques avec la méthode Filter**

In [26]:
model_results_avec_selection_filter = train_and_evaluate_models(X_train_selected_filter, X_test_selected_filter, y_train)
# Afficher les résultats pour chaque modèle
for model_name, result in model_results_avec_selection_filter.items():
    print(f"****************************** Performance Evaluation of {model_name} ****************************************")
    print(classification_report(y_test, result['predictions'], target_names=["Recyclable", "Organique"]))
    print(f"Temps d'exécution {model_name} : {result['time']} seconds")

****************************** Performance Evaluation of SVM ****************************************
              precision    recall  f1-score   support

  Recyclable       0.70      0.73      0.71      2133
   Organique       0.80      0.78      0.79      2931

    accuracy                           0.76      5064
   macro avg       0.75      0.75      0.75      5064
weighted avg       0.76      0.76      0.76      5064

Temps d'exécution SVM : 26.832210779190063 seconds
****************************** Performance Evaluation of Random Forest ****************************************
              precision    recall  f1-score   support

  Recyclable       0.78      0.75      0.77      2133
   Organique       0.82      0.84      0.83      2931

    accuracy                           0.81      5064
   macro avg       0.80      0.80      0.80      5064
weighted avg       0.81      0.81      0.81      5064

Temps d'exécution Random Forest : 6.718001842498779 seconds
*********************

# **************************************************************************************************************************

# 2éme Méthode

#### **1- Extraction des descripteurs :**
* **Méthode Analyse de Texture: "Gabor"** 
*  **techniques de Détection de Contours "Scharr"**

In [27]:
import cv2
import numpy as np
from skimage.filters import gabor
from skimage.feature import local_binary_pattern

# Fonction pour extraire les descripteurs Scharr et Gabor
def extract_features(image):
    # Conversion en niveaux de gris
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # 1. Détection de contours avec les filtres de Scharr
    scharr_x = cv2.Scharr(gray, cv2.CV_64F, 1, 0)  # Dérivée horizontale
    scharr_y = cv2.Scharr(gray, cv2.CV_64F, 0, 1)  # Dérivée verticale
    scharr_magnitude = cv2.magnitude(scharr_x, scharr_y)  # Magnitude des gradients de Scharr
    scharr_hist = np.histogram(scharr_magnitude.ravel(), bins=8, range=(0, 256))[0]  # Histogramme
    scharr_hist = scharr_hist.astype("float")
    scharr_hist /= (scharr_hist.sum() + 1e-6)  # Normalisation
    
    # 2. Analyse de texture avec les filtres de Gabor
    # Application de plusieurs filtres de Gabor à différentes fréquences et orientations
    num_filters = 5  # Nombre de filtres de Gabor à différentes orientations et échelles
    gabor_features = []
    for theta in np.arange(0, np.pi, np.pi / num_filters):  # Différentes orientations
        real, imag = gabor(gray, frequency=0.6, theta=theta)
        gabor_mag = np.sqrt(real**2 + imag**2)
        gabor_hist = np.histogram(gabor_mag.ravel(), bins=8, range=(0, 256))[0]  # Histogramme des magnitudes
        gabor_hist = gabor_hist.astype("float")
        gabor_hist /= (gabor_hist.sum() + 1e-6)  # Normalisation
        gabor_features.append(gabor_hist)
    
    # Concaténer tous les descripteurs (Scharr + Gabor)
    features = np.hstack([scharr_hist] + gabor_features)
    return features

# Aplatir les images en un seul vecteur pour chaque image
def flatten_images(images):
    return images.reshape(images.shape[0], -1)

# Extraire les caractéristiques pour tout l'ensemble de données
def process_dataset(images):
    features = []
    for img in images:
        # Extraction des caractéristiques pour chaque image
        feature = extract_features(img)
        features.append(feature)
    
    # Convertir la liste des caractéristiques en un tableau 2D
    return np.array(features)

# Vérification des formes des données
def check_data_shape(X_train, X_test):
    print(f"Taille des caractéristiques d'entraînement : {X_train.shape}")
    print(f"Taille des caractéristiques de test : {X_test.shape}")


# Extraction des caractéristiques pour l'entraînement et les tests
X_train_features = process_dataset(X_train)
X_test_features = process_dataset(X_test)

# Vérification de la forme des données extraites
check_data_shape(X_train_features, X_test_features)

# Aplatir les caractéristiques
X_train_features_flat_methode3 = flatten_images(X_train_features)
X_test_features_flat_methode3= flatten_images(X_test_features)



Taille des caractéristiques d'entraînement : (11816, 48)
Taille des caractéristiques de test : (5064, 48)


### **2- Entrainements  et évaluations des modéles sans Sélection des caracterstiques**



In [28]:
model_results_methode3 = train_and_evaluate_models(X_train_features_flat_methode3, X_test_features_flat_methode3, y_train)
# Afficher les résultats pour chaque modèle
for model_name, result in model_results_methode3.items():
    print(f"****************************** Performance Evaluation of {model_name} ****************************************")
    print(classification_report(y_test, result['predictions'], target_names=["Recyclable", "Organique"]))
    print(f"Temps d'exécution {model_name} : {result['time']} seconds")

****************************** Performance Evaluation of SVM ****************************************
              precision    recall  f1-score   support

  Recyclable       0.64      0.70      0.67      2133
   Organique       0.77      0.71      0.74      2931

    accuracy                           0.71      5064
   macro avg       0.70      0.71      0.70      5064
weighted avg       0.71      0.71      0.71      5064

Temps d'exécution SVM : 34.83799719810486 seconds
****************************** Performance Evaluation of Random Forest ****************************************
              precision    recall  f1-score   support

  Recyclable       0.71      0.63      0.67      2133
   Organique       0.75      0.81      0.78      2931

    accuracy                           0.73      5064
   macro avg       0.73      0.72      0.72      5064
weighted avg       0.73      0.73      0.73      5064

Temps d'exécution Random Forest : 4.4590065479278564 seconds
*********************

### **3- Sélection de caractéristiques (feature selection)**


**3.1- Sélection par Modèle (Embedded Method) avec RandomForest**

In [29]:
# Sélection par Modèle (Embedded Method) avec RandomForest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_features_flat_methode3, y_train)
# Sélectionner les caractéristiques en fonction de l'importance calculée par RandomForest
select_from_model = SelectFromModel(rf_model, threshold="mean", max_features=20)  # Sélectionner les 20 meilleures caractéristiques
X_train_selected_methode3 = select_from_model.transform(X_train_features_flat_methode3)
X_test_selected_methode3 = select_from_model.transform(X_test_features_flat_methode3)
print(f"Taille des caractéristiques après sélection par modèle : {X_train_selected_methode3.shape}")

Taille des caractéristiques après sélection par modèle : (11816, 8)


**Entrainement  et evaluation des modéles avec la Séléction des caractérstiques avec la méthode embedded**


In [30]:
model_results_avec_selection = train_and_evaluate_models(X_train_selected_methode3, X_test_selected_methode3, y_train)
# Afficher les résultats pour chaque modèle
for model_name, result in model_results_avec_selection.items():
    print(f"****************************** Performance Evaluation of {model_name} ****************************************")
    print(classification_report(y_test, result['predictions'], target_names=["Recyclable", "Organique"]))
    print(f"Temps d'exécution {model_name} : {result['time']} seconds")

****************************** Performance Evaluation of SVM ****************************************
              precision    recall  f1-score   support

  Recyclable       0.64      0.70      0.67      2133
   Organique       0.77      0.71      0.74      2931

    accuracy                           0.71      5064
   macro avg       0.70      0.71      0.70      5064
weighted avg       0.71      0.71      0.71      5064

Temps d'exécution SVM : 30.72598171234131 seconds
****************************** Performance Evaluation of Random Forest ****************************************
              precision    recall  f1-score   support

  Recyclable       0.71      0.63      0.67      2133
   Organique       0.75      0.81      0.78      2931

    accuracy                           0.73      5064
   macro avg       0.73      0.72      0.72      5064
weighted avg       0.73      0.73      0.73      5064

Temps d'exécution Random Forest : 4.8829731941223145 seconds
*********************

**3.2 - Sélection par Test Statistique (Filter Method) avec SelectKBest et ANOVA F-test**

In [31]:
# Sélection par Test Statistique (Filter Method) avec SelectKBest et ANOVA F-test
selector = SelectKBest(f_classif, k=20)  # Sélectionner les 20 meilleures caractéristiques
X_train_selected_filter = selector.fit_transform(X_train_features_flat_methode3, y_train)
X_test_selected_filter = selector.transform(X_test_features_flat_methode3)
print(f"Taille des caractéristiques après sélection par test statistique : {X_train_selected_filter.shape}")

Taille des caractéristiques après sélection par test statistique : (11816, 20)


c:\Users\debgn\anaconda3\envs\ml_env\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 9 10 11 12 13 14 15 17 18 19 20 21 22 23 25 26 27 28 29 30 31 33 34 35
 36 37 38 39 41 42 43 44 45 46 47] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\debgn\anaconda3\envs\ml_env\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


**Entrainement  et evaluation des modéles avec la Séléction des caractérstiques avec la méthode Filter**

In [32]:
model_results_avec_selection_filter = train_and_evaluate_models(X_train_selected_filter, X_test_selected_filter, y_train)
# Afficher les résultats pour chaque modèle
for model_name, result in model_results_avec_selection_filter.items():
    print(f"****************************** Performance Evaluation of {model_name} ****************************************")
    print(classification_report(y_test, result['predictions'], target_names=["Recyclable", "Organique"]))
    print(f"Temps d'exécution {model_name} : {result['time']} seconds")

****************************** Performance Evaluation of SVM ****************************************
              precision    recall  f1-score   support

  Recyclable       0.64      0.70      0.67      2133
   Organique       0.77      0.71      0.74      2931

    accuracy                           0.71      5064
   macro avg       0.70      0.71      0.70      5064
weighted avg       0.71      0.71      0.71      5064

Temps d'exécution SVM : 31.833030939102173 seconds
****************************** Performance Evaluation of Random Forest ****************************************
              precision    recall  f1-score   support

  Recyclable       0.70      0.63      0.66      2133
   Organique       0.75      0.81      0.78      2931

    accuracy                           0.73      5064
   macro avg       0.72      0.72      0.72      5064
weighted avg       0.73      0.73      0.73      5064

Temps d'exécution Random Forest : 4.54604434967041 seconds
**********************

## **************************************************************************************************************************

# 3éme Méthode (ajout d'un descripteur de couleur)

#### **1- Extraction des descripteurs :**
* **Méthode Analyse de Texture: "GLCM"** 
*  **techniques de Détection de Contours "Canny"**
* **Histogramme de couleurs (HSV) pour extraire les couleur**

In [33]:
import numpy as np
import cv2
from skimage.feature import  graycomatrix, graycoprops

# Fonction pour extraire les descripteurs (GLCM, HSV, et Canny)
def extract_features(image):
    # Conversion en niveaux de gris
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # 1. Canny (Contours)
    edges = cv2.Canny(gray, threshold1=100, threshold2=200)
    edges_hist = np.histogram(edges.ravel(), bins=2, range=(0, 2))[0]  # Histogramme des bords
    edges_hist = edges_hist.astype("float")
    edges_hist /= (edges_hist.sum() + 1e-6)  # Normalisation
    
    # 2. Histogramme de couleurs (HSV)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256]).flatten()
    hist /= (hist.sum() + 1e-6)  # Normalisation
    
    # 3. GLCM (Gray-Level Co-occurrence Matrix) pour la texture
    glcm = graycomatrix(gray, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    
    # Concaténer les descripteurs
    features = np.hstack([hist, edges_hist, contrast, homogeneity])
    return features

# Fonction pour aplatir les données
def flatten_images(images):
    return images.reshape(images.shape[0], -1)

# Extraire les caractéristiques pour tout l'ensemble de données
def process_dataset(images):
    features = []
    for img in images:
        # Extraction des caractéristiques pour chaque image
        feature = extract_features(img)
        features.append(feature)
    
    # Convertir la liste des caractéristiques en un tableau 2D
    return np.array(features)

# Vérification des formes des données
def check_data_shape(X_train, X_test):
    print(f"Taille des caractéristiques d'entraînement : {X_train.shape}")
    print(f"Taille des caractéristiques de test : {X_test.shape}")



In [34]:
# Extraction des caractéristiques pour l'entraînement et les tests
X_train_features = process_dataset(X_train)
X_test_features = process_dataset(X_test)

# Vérification de la forme des données extraites
check_data_shape(X_train_features, X_test_features)

# Aplatir les caractéristiques (si nécessaire)
X_train_features = flatten_images(X_train_features)
X_test_features = flatten_images(X_test_features)

# Vérification des formes après aplatissement
print("********************* apres flatten ***********************")
check_data_shape(X_train_features, X_test_features)


Taille des caractéristiques d'entraînement : (11816, 516)
Taille des caractéristiques de test : (5064, 516)
********************* apres flatten ***********************
Taille des caractéristiques d'entraînement : (11816, 516)
Taille des caractéristiques de test : (5064, 516)


### **2-Entrainements  et évaluations des modéles sans Sélection des caracterstiques**

In [47]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import time
from sklearn.ensemble import RandomForestClassifier

from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb



# 1. Modèle KNN (K-Nearest Neighbors)
knn1 = KNeighborsClassifier(n_neighbors=5)
# Entraînement des modèles
knn_time_start= time.time()
knn1.fit(X_train_features, y_train)
knn_time_end= time.time()
# Faire des prédictions
pred_knn = knn1.predict(X_test_features)


# 2. Modèle RandomForest
random_forest1 = RandomForestClassifier(n_estimators=100, random_state=42)
# Entraînement des modèles
rd_time_start= time.time()
random_forest1.fit(X_train_features, y_train)
rd_time_end= time.time()
# Faire des prédictions
pred_rf = random_forest1.predict(X_test_features)



# 3. Modèle XGBoost
xgb_model1 = xgb.XGBClassifier(n_estimators=100, random_state=42)
# Entraînement des modèles
xgb_time_start= time.time()
xgb_model1.fit(X_train_features, y_train)
xgb_time_end= time.time()
# Faire des prédictions
pred_xgb = xgb_model1.predict(X_test_features)



In [48]:

print("****************************** Performance Evaluation of Random Forest ****************************************")
print(classification_report(y_test, pred_rf, target_names=["Recyclable", "Organique"]))
print("le temp d'execution :",rd_time_end-rd_time_start)


print("****************************** Performance Evaluation of  XGboost ****************************************")
print(classification_report(y_test, pred_xgb, target_names=["Recyclable", "Organique"]))
print("le temp d'execution :",xgb_time_end-xgb_time_start)


print("****************************** Performance Evaluation of KNN ****************************************")
print(classification_report(y_test, pred_knn, target_names=["Recyclable", "Organique"]))
print("le temp d'execution des:",knn_time_end-knn_time_start)

****************************** Performance Evaluation of Random Forest ****************************************
              precision    recall  f1-score   support

  Recyclable       0.88      0.80      0.84      2133
   Organique       0.86      0.92      0.89      2931

    accuracy                           0.87      5064
   macro avg       0.87      0.86      0.86      5064
weighted avg       0.87      0.87      0.87      5064

le temp d'execution : 10.53600001335144
****************************** Performance Evaluation of  XGboost ****************************************
              precision    recall  f1-score   support

  Recyclable       0.87      0.82      0.85      2133
   Organique       0.88      0.91      0.89      2931

    accuracy                           0.87      5064
   macro avg       0.87      0.87      0.87      5064
weighted avg       0.87      0.87      0.87      5064

le temp d'execution : 5.494972467422485
****************************** Performance Eval

### **3- Sélection de caractéristiques (feature selection)**


**3.1- Sélection par Modèle (Embedded Method) avec RandomForest**

In [37]:
# Récupérer l'importance des caractéristiques du model random forest
importances = random_forest1.feature_importances_

# Trier les caractéristiques selon leur importance
indices = np.argsort(importances)[::-1]  # Trier par ordre décroissant
X_train_selected1 = X_train_features[:, indices[:50]]  # Garder les 10 meilleures caractéristiques
X_test_selected1= X_test_features[:, indices[:50]]  # Garder les 10 meilleures caractéristiques

print(f"Taille des caractéristiques de train apres selection des caractéeistique : {X_train_selected1.shape}")
print(f"Taille des caractéristiques de test apres selection des caractéeistique : {X_test_selected1.shape}")

Taille des caractéristiques de train apres selection des caractéeistique : (11816, 50)
Taille des caractéristiques de test apres selection des caractéeistique : (5064, 50)


**Apprentissage apres selection des caractéristiques**

In [38]:
# 1. Modèle RandomForest
random_forest2 = RandomForestClassifier(n_estimators=100, random_state=42)

# 2. Modèle SVC (Support Vector Machine)

# 3. Modèle XGBoost

xgb_model2 = xgb.XGBClassifier(n_estimators=100, random_state=42)

# 4. Modèle KNN (K-Nearest Neighbors)
knn2 = KNeighborsClassifier(n_neighbors=5)

# Entraînement des modèles
rd_time_start= time.time()
random_forest2.fit(X_train_selected1, y_train)
rd_time_end= time.time()


xgb_time_start= time.time()
xgb_model2.fit(X_train_selected1, y_train)
xgb_time_end= time.time()

knn_time_start= time.time()
knn2.fit(X_train_selected1, y_train)
knn_time_end= time.time()



# Faire des prédictions

pred_rf = random_forest2.predict(X_test_selected1)
pred_xgb = xgb_model2.predict(X_test_selected1)
pred_knn = knn2.predict(X_test_selected1)



**Evaluation**

In [39]:
# Evaluation


print("****************************** Performance Evaluation of Random Forest ****************************************")
print(classification_report(y_test, pred_rf, target_names=["Recyclable", "Organique"]))
print("le temp d'execution :",rd_time_end-rd_time_start)


print("****************************** Performance Evaluation of  XGboost ****************************************")
print(classification_report(y_test, pred_xgb, target_names=["Recyclable", "Organique"]))
print("le temp d'execution :",xgb_time_end-xgb_time_start)


print("****************************** Performance Evaluation of KNN ****************************************")
print(classification_report(y_test, pred_knn, target_names=["Recyclable", "Organique"]))
print("le temp d'execution des:",knn_time_end-knn_time_start)



****************************** Performance Evaluation of Random Forest ****************************************
              precision    recall  f1-score   support

  Recyclable       0.85      0.78      0.81      2133
   Organique       0.85      0.90      0.87      2931

    accuracy                           0.85      5064
   macro avg       0.85      0.84      0.84      5064
weighted avg       0.85      0.85      0.85      5064

le temp d'execution : 6.359045743942261
****************************** Performance Evaluation of  XGboost ****************************************
              precision    recall  f1-score   support

  Recyclable       0.84      0.80      0.82      2133
   Organique       0.86      0.89      0.87      2931

    accuracy                           0.85      5064
   macro avg       0.85      0.84      0.85      5064
weighted avg       0.85      0.85      0.85      5064

le temp d'execution : 0.9649510383605957
****************************** Performance Eva

**3.2 - Sélection par Test Statistique (Filter Method) avec SelectKBest et ANOVA F-test**

In [40]:
from sklearn.feature_selection import SelectKBest, f_classif

# Utiliser ANOVA pour sélectionner les meilleures caractéristiques
selector = SelectKBest(f_classif, k=50)  # Vous pouvez ajuster k pour sélectionner un nombre spécifique de caractéristiques
X_train_selected2 = selector.fit_transform(X_train_features, y_train)
X_test_selected2 = selector.transform(X_test_features)



anova_scores = selector.scores_
print(f"Taille des caractéristiques de train apres selection des caractéeistique : {X_train_selected2.shape}")
print(f"Taille des caractéristiques de test apres selection des caractéeistique : {X_test_selected2.shape}")




Taille des caractéristiques de train apres selection des caractéeistique : (11816, 50)
Taille des caractéristiques de test apres selection des caractéeistique : (5064, 50)


c:\Users\debgn\anaconda3\envs\ml_env\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [384 385 386 387 388 389 390 391 392 393 394 395 396 397 398 399 400 401
 402 403 404 405 406 407 408 409 410 411 412 413 414 415 416 417 418 419
 420 421 422 423 424 425 426 427 428 429 430 431 432 433 434 435 436 437
 438 439 440 441 442 443 444 445 446 447 448 449 450 451 452 453 454 455
 456 457 458 459 460 461 462 463 464 465 466 467 468 469 470 471 472 473
 474 475 476 477 478 479 480 481 482 483 484 485 486 487 488 489 490 491
 492 493 494 495 496 497 498 499 500 501 502 503 504 505 506 507 508 509
 510 511 513] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\debgn\anaconda3\envs\ml_env\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


**Apprentissage apres selection des caractéristiques**

In [45]:
# 1. Modèle RandomForest
random_forest3 = RandomForestClassifier(n_estimators=100, random_state=42)

# 2. Modèle SVC (Support Vector Machine)

# 3. Modèle XGBoost

xgb_model3 = xgb.XGBClassifier(n_estimators=100, random_state=42)

# 4. Modèle KNN (K-Nearest Neighbors)
knn3 = KNeighborsClassifier(n_neighbors=5)

# Entraînement des modèles
rd_time_start= time.time()
random_forest3.fit(X_train_selected2, y_train)
rd_time_end= time.time()



xgb_time_start= time.time()
xgb_model3.fit(X_train_selected2, y_train)
xgb_time_end= time.time()

knn_time_start= time.time()
knn3.fit(X_train_selected2, y_train)
knn_time_end= time.time()



# Faire des prédictions

pred_rf = random_forest3.predict(X_test_selected2)
pred_xgb = xgb_model3.predict(X_test_selected2)
pred_knn = knn3.predict(X_test_selected2)



**Evaluation**

In [46]:
# Evaluation


print("****************************** Performance Evaluation of Random Forest ****************************************")
print(classification_report(y_test, pred_rf, target_names=["Recyclable", "Organique"]))
print("le temp d'execution :",rd_time_end-rd_time_start)


print("****************************** Performance Evaluation of  XGboost ****************************************")
print(classification_report(y_test, pred_xgb, target_names=["Recyclable", "Organique"]))
print("le temp d'execution :",xgb_time_end-xgb_time_start)


print("****************************** Performance Evaluation of KNN ****************************************")
print(classification_report(y_test, pred_knn, target_names=["Recyclable", "Organique"]))
print("le temp d'execution des:",knn_time_end-knn_time_start)



****************************** Performance Evaluation of Random Forest ****************************************
              precision    recall  f1-score   support

  Recyclable       0.85      0.77      0.81      2133
   Organique       0.84      0.90      0.87      2931

    accuracy                           0.85      5064
   macro avg       0.85      0.84      0.84      5064
weighted avg       0.85      0.85      0.85      5064

le temp d'execution : 6.566993474960327
****************************** Performance Evaluation of  XGboost ****************************************
              precision    recall  f1-score   support

  Recyclable       0.83      0.79      0.81      2133
   Organique       0.85      0.89      0.87      2931

    accuracy                           0.84      5064
   macro avg       0.84      0.84      0.84      5064
weighted avg       0.84      0.84      0.84      5064

le temp d'execution : 1.1289989948272705
****************************** Performance Eva

# Conclusion 
Dans le cadre de ce travail, nous avons exploré trois combinaisons distinctes de descripteurs, en utilisant deux approches différentes pour la sélection des caractéristiques : la sélection **basée sur les modèles** et la sélection basée sur des **tests statistiques**.

**Points clés :**
* **Impact de la sélection des caractéristiques :**
La sélection des caractéristiques s'est avérée bénéfique, non seulement pour améliorer **la précision** de la classification, mais également pour réduire considérablement le **temps d'apprentissage** des modèles. Cela a permis de diminuer la complexité des données tout en préservant les informations essentielles.

* **Importance des descripteurs de couleur :**
Nous avons constaté une amélioration significative des performances lorsque nous avons intégré un descripteur de couleur, notamment l'histogramme en espace de couleurs HSV. Ce descripteur a enrichi la représentation des images en capturant des informations pertinentes sur les nuances et intensités, essentielles pour différencier les classes.


L'optimisation des caractéristiques, tant par le choix des descripteurs que par leur sélection stratégique, est une étape cruciale pour obtenir des modèles performants et efficients. Ce travail met en évidence l'importance d'une approche combinée, en tirant parti de la complémentarité entre descripteurs de texture, de contour et de couleur, pour maximiser les résultats.